In [1]:
#Import necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv(r"C:\Users\Felipe\Football_Analytics_Portfolio\Libertadores_Process_Dominance\data\libertadores_champs.csv")

In [13]:
#Initial inspection
print(df.shape)
print(df['Round'].value_counts())
print(df.columns.tolist())

(150, 10)
Round
Group stage                54
Round of 16                22
Quarter-finals             22
Semi-finals                20
Second stage               12
Finals                      8
Final                       6
Second qualifying stage     2
Third qualifying stage      2
Semi-Finals                 2
Name: count, dtype: int64
['Date', 'Round', 'Venue', 'Result', 'GF', 'GA', 'Opponent', 'Poss', 'xG', 'xGA']


In [14]:
# Extract year from the date column and store it in a new column
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [15]:
#Check frequency for each year
df['Year'].value_counts().sort_index()

Year
2015    14
2016    14
2017    14
2018    14
2019    13
2020    10
2021    16
2022    13
2023    13
2024    17
2025    12
Name: count, dtype: int64

In [16]:
## Create the champion mapping dictionary
champion_map = {
    2015: 'River Plate',
    2016: 'Atlético Nacional', 
    2017: 'Grêmio',
    2018: 'River Plate',
    2019: 'Flamengo',
    2020: 'Palmeiras',
    2021: 'Palmeiras',
    2022: 'Flamengo',
    2023: 'Fluminense',
    2024: 'Botafogo',
    2025: 'Palmeiras*'  # The asterisk flags it as ongoing
}

In [17]:
# Test
print(champion_map[2023])  # Should print: Fluminense*
print(champion_map[2019])  # Should print: Flamengo

Fluminense
Flamengo


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      150 non-null    object 
 1   Round     150 non-null    object 
 2   Venue     150 non-null    object 
 3   Result    150 non-null    object 
 4   GF        150 non-null    int64  
 5   GA        150 non-null    int64  
 6   Opponent  150 non-null    object 
 7   Poss      122 non-null    float64
 8   xG        94 non-null     float64
 9   xGA       94 non-null     float64
 10  Year      150 non-null    int32  
dtypes: float64(3), int32(1), int64(2), object(5)
memory usage: 12.4+ KB


In [19]:
#Check for complete empty rows
print("Rows with missing Date:")
print(df[df['Year'].isna()])

Rows with missing Date:
Empty DataFrame
Columns: [Date, Round, Venue, Result, GF, GA, Opponent, Poss, xG, xGA, Year]
Index: []


In [20]:
# Convert Date to datetime and save it
df['Date'] = pd.to_datetime(df['Date'])
print("Checking if date column converted to datetime")
df.info()

Checking if date column converted to datetime
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      150 non-null    datetime64[ns]
 1   Round     150 non-null    object        
 2   Venue     150 non-null    object        
 3   Result    150 non-null    object        
 4   GF        150 non-null    int64         
 5   GA        150 non-null    int64         
 6   Opponent  150 non-null    object        
 7   Poss      122 non-null    float64       
 8   xG        94 non-null     float64       
 9   xGA       94 non-null     float64       
 10  Year      150 non-null    int32         
dtypes: datetime64[ns](1), float64(3), int32(1), int64(2), object(4)
memory usage: 12.4+ KB


In [21]:
# Apply the mappingb
df['Champion'] = df['Year'].map(champion_map)

In [22]:
# Convert Year to integer
df['Year'] = df['Year'].astype(int)

In [23]:
# Check all unique Round values
print("Unique Round values:")
print(df['Round'].unique())
print("\nRound value counts:")
print(df['Round'].value_counts())

Unique Round values:
['Second stage' 'Round of 16' 'Quarter-finals' 'Semi-finals' 'Finals'
 'Group stage' 'Final' 'Second qualifying stage' 'Third qualifying stage'
 'Semi-Finals']

Round value counts:
Round
Group stage                54
Round of 16                22
Quarter-finals             22
Semi-finals                20
Second stage               12
Finals                      8
Final                       6
Second qualifying stage     2
Third qualifying stage      2
Semi-Finals                 2
Name: count, dtype: int64


In [24]:
# Create a mapping dictionary
stage_map = {
    'Second qualifying stage': 'Qualifying',
    'Third qualifying stage': 'Qualifying',
    'Second stage': 'Group',
    'Group stage': 'Group',
    'Round of 16': 'R16',
    'Quarter-finals': 'QF',
    'Semi-finals': 'SF',
    'Finals': 'Final',
    'Final': 'Final'
}

# Apply the mapping
df['Stage'] = df['Round'].map(stage_map)

In [25]:
# Verify it worked
print("New Stage column:")
print(df['Stage'].value_counts())

New Stage column:
Stage
Group         66
R16           22
QF            22
SF            20
Final         14
Qualifying     4
Name: count, dtype: int64


In [26]:
#Filter qualifying rounds out of the main dataframe
df = df.query('Stage != "Qualifying"')

In [27]:
# Verify
print(f"Total matches after removing Qualifying: {len(df)}")
print("\nStage distribution:")
print(df['Stage'].value_counts())
print("\nMatches per champion:")
print(df.groupby('Champion').size().sort_index())

Total matches after removing Qualifying: 146

Stage distribution:
Stage
Group    66
R16      22
QF       22
SF       20
Final    14
Name: count, dtype: int64

Matches per champion:
Champion
Atlético Nacional    14
Botafogo             13
Flamengo             26
Fluminense           13
Grêmio               14
Palmeiras            26
Palmeiras*           12
River Plate          28
dtype: int64


In [28]:
# Check unique Result values
print("Unique Result values:")
print(df['Result'].unique())
print("\nResult value counts:")
print(df['Result'].value_counts())

Unique Result values:
['L' 'D' 'W']

Result value counts:
Result
W    95
D    34
L    17
Name: count, dtype: int64


In [29]:
# Find matches that might have gone to penalties
# (Knockout matches that ended in draws are candidates)

knockout_draws = df.query('Stage in ["R16", "QF", "SF"] and Result == "D"')
print(f"Knockout draws (potential penalty matches): {len(knockout_draws)}")
print(knockout_draws[['Date', 'Champion', 'Stage', 'Result', 'GF', 'GA', 'Opponent']])

Knockout draws (potential penalty matches): 17
          Date           Champion Stage Result  GF  GA             Opponent
11  2015-07-21        River Plate    SF      D   1   1           py Guaraní
20  2016-04-26  Atlético Nacional   R16      D   0   0           ar Huracán
36  2017-09-13             Grêmio    QF      D   0   0     br Botafogo (RJ)
48  2018-08-09        River Plate   R16      D   0   0       ar Racing Club
50  2018-09-19        River Plate    QF      D   0   0     ar Independiente
65  2019-08-28           Flamengo    QF      D   1   1     br Internacional
66  2019-10-02           Flamengo    SF      D   1   1            br Grêmio
77  2020-12-08          Palmeiras    QF      D   1   1          py Libertad
90  2021-08-10          Palmeiras    QF      D   1   1         br São Paulo
92  2021-09-21          Palmeiras    SF      D   0   0  br Atlético Mineiro
93  2021-09-28          Palmeiras    SF      D   1   1  br Atlético Mineiro
114 2023-08-01         Fluminense   R16  

In [30]:
def extract_goals(value):
    if pd.isna(value):
        return None
    # Convert to string and get first part before any parenthesis
    clean_value = str(value).split('(')[0].strip()
    # Convert to float first, then int (handles "2.0")
    return int(float(clean_value))

In [31]:
# Apply to both columns
df['GF'] = df['GF'].apply(extract_goals)
df['GA'] = df['GA'].apply(extract_goals)

In [32]:
# Verify
print("GF/GA data types:")
print(df[['GF', 'GA']].dtypes)
print("\nSample data:")
print(df[['Date', 'Champion', 'Result', 'GF', 'GA', 'Opponent']].tail(10))

GF/GA data types:
GF    int64
GA    int64
dtype: object

Sample data:
          Date    Champion Result  GF  GA          Opponent
140 2025-04-24  Palmeiras*      W   3   2           Bolivar
141 2025-05-07  Palmeiras*      W   2   0     Cerro Porteño
142 2025-05-15  Palmeiras*      W   2   0           Bolivar
143 2025-05-28  Palmeiras*      W   6   0  Sporting Cristal
144 2025-08-14  Palmeiras*      W   4   0     Universitario
145 2025-08-21  Palmeiras*      D   0   0     Universitario
146 2025-09-17  Palmeiras*      W   2   1       River Plate
147 2025-09-24  Palmeiras*      W   3   1       River Plate
148 2025-10-23  Palmeiras*      L   0   3               LDU
149 2025-10-30  Palmeiras*      W   4   0               LDU


In [33]:
# Final data check
print("=== CLEANED DATASET SUMMARY ===\n")
print(f"Total matches: {len(df)}")
print(f"\nChampions: {df['Champion'].nunique()}")
print(f"Years: {sorted(df['Year'].unique())}")
print(f"\nStages: {df['Stage'].unique()}")
print(f"\nData types:")
print(df.dtypes)
print(f"\nNull values:")
print(df.isnull().sum())

=== CLEANED DATASET SUMMARY ===

Total matches: 146

Champions: 8
Years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]

Stages: ['Group' 'R16' 'QF' 'SF' 'Final' nan]

Data types:
Date        datetime64[ns]
Round               object
Venue               object
Result              object
GF                   int64
GA                   int64
Opponent            object
Poss               float64
xG                 float64
xGA                float64
Year                 int64
Champion            object
Stage               object
dtype: object

Null values:
Date         0
Round        0
Venue        0
Result       0
GF           0
GA           0
Opponent     0
Poss        28
xG          56
xGA         56
Year         0
Champion     0
Stage        2
dtype: int64


In [34]:
# Check which years have missing xG data
print("xG availability by year:")
print(df.groupby('Year')['xG'].apply(lambda x: x.notna().sum()))
print(f"\nTotal pre-2019 matches: {len(df[df['Year'] < 2019])}")
print(f"Total null xG values: {df['xG'].isna().sum()}")

xG availability by year:
Year
2015     0
2016     0
2017     0
2018     0
2019    13
2020    10
2021    16
2022    13
2023    13
2024    13
2025    12
Name: xG, dtype: int64

Total pre-2019 matches: 56
Total null xG values: 56


In [37]:
df.to_parquet(r'C:\Users\Felipe\Football_Analytics_Portfolio\Libertadores_Process_Dominance\data\clean_data.parquet', index=False)